<a href="https://colab.research.google.com/github/VIVEKPATIL12/U-Net-Segmentation/blob/main/U_net_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/aamirmiy/Boreas.git

Cloning into 'Boreas'...
remote: Enumerating objects: 3008, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3008 (delta 0), reused 3 (delta 0), pack-reused 3005 (from 1)
Receiving objects: 100% (3008/3008), 190.54 MiB | 18.86 MiB/s, done.


In [ ]:

import cv2
import numpy as np
import os

class ROICalculator:
    def __init__(self, original_size, new_size, focal_length, principal_points, radial_distortion):
        self.original_size = np.array(original_size)
        self.new_size = np.array(new_size)
        self.scale_factor = self.new_size / self.original_size
        self.focal_length = np.array(focal_length) * self.scale_factor[0]
        self.principal_points = np.array(principal_points) * self.scale_factor
        self.radial_distortion = np.array(radial_distortion)

        # Create camera matrix
        self.camera_matrix = np.array([
            [self.focal_length[0], 0, self.principal_points[0]],
            [0, self.focal_length[1], self.principal_points[1]],
            [0, 0, 1]
        ])

        # Create distortion coefficients
        self.dist_coeffs = np.array([self.radial_distortion[0], self.radial_distortion[1], 0, 0, 0])



In [ ]:
  def calculate_road_line(self, distance, height_above_ground):
        # Define 3D points of a line on the road at the given distance
        road_width = 4  # Adjust based on typical road width
        world_points = np.array([
            [-road_width/2, 0, distance],  # Left point on road
            [road_width/2, 0, distance]    # Right point on road
        ], dtype=np.float32)

        # Define camera pose (assuming camera is looking straight ahead)

        rvec = np.array([0, 0, 0], dtype=np.float32)
        #rvec = np.array([0, 0, 0], dtype=np.float32)
        tvec = np.array([0, height_above_ground, 0], dtype=np.float32)

        # Project 3D points to 2D image plane
        image_points, _ = cv2.projectPoints(world_points, rvec, tvec, self.camera_matrix, self.dist_coeffs)

        # Extract the line endpoints
        left_point = tuple(map(int, image_points[0][0]))
        right_point = tuple(map(int, image_points[1][0]))

        return left_point, right_point



In [ ]:
def process_single_image_road_lines(image_path, roi_calculator, output_folder, start_distance, end_distance, interval):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Read image
    frame = cv2.imread(image_path)

    if frame is None:
        print(f"Unable to read image: {image_path}")
        return None

    lines = []

    # Calculate road lines at different distances
    for distance in range(start_distance, end_distance + 1, interval):
        left_point, right_point = roi_calculator.calculate_road_line(distance, height_above_ground=1.5)
        lines.append((distance, left_point, right_point))

        # Draw line on road
        color = (0, 255, 0)  # Green color
        cv2.line(frame, left_point, right_point, color, 2)

    # Save processed image
    image_filename = os.path.basename(image_path)
    output_path = os.path.join(output_folder, f"processed_road_lines_{image_filename}")
    cv2.imwrite(output_path, frame)

    print(f"Processed image: Road lines calculated from {start_distance}m to {end_distance}m at {interval}m intervals")

    return frame, lines

In [ ]:
# Example usage
original_size = [2448, 2048]
new_size = [224, 224]
focal_length = [1436.89437780124, 1442.58498490709]
principal_points = [1218.77863491507, 1045.77362896196]
radial_distortion = [0.0144210402954516, -0.0148287624062537]

roi_calc = ROICalculator(original_size, new_size, focal_length, principal_points, radial_distortion)

# Process a single image with road lines
image_path = '/content/1616517895573662.png'
output_folder = '/content/outputs'
start_distance = 5  # Starting distance in meters
end_distance = 15   # Ending distance in meters
interval = 5      # Interval between lines in meters


processed_frame, calculated_lines = process_single_image_road_lines(image_path, roi_calc, output_folder, start_distance, end_distance, interval)

if processed_frame is not None:
    # Display the processed image (optional)
    processed_frame

    print("Calculated Road Lines:")
    for distance, left_point, right_point in calculated_lines:
        print(f"Distance: {distance}m, Left: {left_point}, Right: {right_point}")

Processed image: Road lines calculated from 5m to 15m at 5m intervals
Calculated Road Lines:
Distance: 5m, Left: (58, 154), Right: (164, 154)
Distance: 10m, Left: (85, 134), Right: (137, 134)
Distance: 15m, Left: (93, 127), Right: (129, 127)
